# Souvenirs Exercise
### Import Data

In [1]:
import pandas as pd
import os

souvenirs_df = pd.read_csv('datasets/souvenir/souvenir.csv')
category_df = pd.read_csv('datasets/souvenir/category.csv')
city_df = pd.read_csv('datasets/souvenir/city.csv')
country_df = pd.read_csv('datasets/souvenir/country.csv')
location_df = pd.read_csv('datasets/souvenir/location.csv')
owner_df = pd.read_csv('datasets/souvenir/owner.csv')
region_df = pd.read_csv('datasets/souvenir/region.csv')

print(souvenirs_df.shape[0])

147


### Combine DataFrames and Change the name of the IDS

In [2]:
# Rename similar columns to prepare for merge
category_df.rename(columns = {'Name': 'CategoryName'}, inplace = True)
city_df.rename(columns = {'Name': 'City'}, inplace = True)
region_df.rename(columns = {'Name': 'Region'}, inplace = True)
country_df.rename(columns = {'name': 'Country'}, inplace = True)
owner_df.rename(columns = {'Name': 'Owner'}, inplace = True)

# Merge location frames
city_region = pd.merge(city_df, region_df, left_on = 'RegionID', right_on = 'RegionID')
location = pd.merge(city_region, country_df, left_on = 'CountryID', right_on = 'CountryID')

# Slim down the location frame
location = location[['CityID', 'City', 'Region', 'Country']]

# Slim down the second location frame
locale_df = location_df[['LocationID', 'PlaceName']]

# Merge Primary Df with secondary
souvenirs_df = pd.merge(souvenirs_df, category_df, left_on = 'CategoryID', right_on = 'CategoryID', how = 'left')
souvenirs_df = pd.merge(souvenirs_df, location, left_on = 'CityID', right_on = 'CityID', how = 'left')
souvenirs_df = pd.merge(souvenirs_df, locale_df, left_on = 'LocationID', right_on = 'LocationID', how = 'left')
souvenirs_df = pd.merge(souvenirs_df, owner_df, left_on = 'OwnerID', right_on = 'OwnerID', how = 'left')

# Define which columns to keep
columns_to_keep = [_ for _ in souvenirs_df.columns.values.tolist() if 'ID' not in _]

# Slim down the resultant dataframe
souvenirs_df = souvenirs_df[[_ for _ in columns_to_keep]]

souvenirs_df.head()

,ItemDescription,DateObtained,Price,Weight,Name,CategoryName,City,Region,Country,PlaceName,Owner
0,NaN,2019/02/15,7.50,12.0,Grizzly Bear Mug,Kitchenware,NaN,NaN,NaN,Rocky Mountain National Park,Group 3
1,NaN,2019/02/17,9.99,20.0,Snowy Mountain Mug,Kitchenware,NaN,NaN,NaN,Rocky Mountain National Park,Group 3
2,NaN,2019/03/05,12.99,16.0,Lobster Mug,Kitchenware,NaN,NaN,NaN,Offshore Trap/Pot Waters Area,Group 3
3,NaN,2019/04/18,10.25,16.0,Sea Lion Mug,Kitchenware,NaN,NaN,NaN,Steller Sea Lion Protection Area,Group 3
4,NaN,2019/05/07,6.91,20.0,Coral Reef Mug,Kitchenware,NaN,NaN,NaN,Pacific Remote Islands Marine National Monument,Group 3


### Export to CSV

In [3]:
for owner in list(set(souvenirs_df.Owner.tolist())):
    souvenirs_df[(souvenirs_df.Owner == owner)].to_csv(f'souvenir_output/{owner}.csv', index = False)
    